<a href="https://colab.research.google.com/github/Junaeid-Shoaib/AI_Agents/blob/main/Viz_LlamaIndex_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface -U -q

In [2]:
%pip install -U jedi



In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('secretKey')

In [4]:
from huggingface_hub import login

login()

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from google.colab import drive
drive.mount('/content/drive')

# Path to your folder inside Google Drive
folder_path = '/content/drive/MyDrive/Colab/AI_Agents/state_of_ai.pdf'
# Load document
reader = SimpleDirectoryReader(input_files=[folder_path]) #input_files=["state_of_ai.pdf"]
documents = reader.load_data()
print(f"Loaded {len(documents)} document(s).")

# Split into chunks
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

# Set up LLM and embedding model
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Create vector index
vector_index = VectorStoreIndex(nodes)

# Create query engine
query_engine = vector_index.as_query_engine()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 26 document(s).


**Inspecting the vector store**

In [6]:
# Access the vector store data directly
vector_store = vector_index.vector_store

# Get embedding dictionary and node dictionary
embedding_dict = vector_store.data.embedding_dict
node_dict = vector_store.data.text_id_to_ref_doc_id

print(f"Number of embeddings: {len(embedding_dict)}")
print(f"Number of node references: {len(node_dict)}")

# Show first few embeddings
for i, (node_id, embedding) in enumerate(list(embedding_dict.items())[:3]):
    print(f"\n--- Embedding {i} ---")
    print(f"Node ID: {node_id}")
    print(f"Embedding dimension: {len(embedding)}")
    print(f"First 10 values: {embedding[:10]}")

Number of embeddings: 27
Number of node references: 27

--- Embedding 0 ---
Node ID: 7656f019-83ad-4199-9c76-c378faea308f
Embedding dimension: 1536
First 10 values: [-0.010264999233186245, -0.014738407917320728, -0.008932793512940407, -0.02048792876303196, 0.006773216649889946, 0.007867028005421162, -0.019842861220240593, 0.0186088178306818, -0.029308537021279335, -0.022927969694137573]

--- Embedding 1 ---
Node ID: b69be3a3-bf22-4fcd-a7dc-491b79c0d1bd
Embedding dimension: 1536
First 10 values: [-0.014604520983994007, -0.020579729229211807, -0.0003736676590051502, -0.019204042851924896, -0.0048461719416081905, 0.015132563188672066, -0.025846250355243683, 0.013277469202876091, -0.022191647440195084, -0.02649935521185398]

--- Embedding 2 ---
Node ID: 4d28b21f-2d7c-48d5-9d72-1ce0ec882d09
Embedding dimension: 1536
First 10 values: [0.004264378920197487, -0.029195113107562065, -0.006927081383764744, -0.03330405429005623, -0.01008988544344902, 0.012522811070084572, -0.013766306452453136, 0.

**Using rag to get answer**

In [7]:
# Query the document
response = query_engine.query("Who is Lareina Yee?")
print(response)

Lareina Yee is a Senior partner and McKinsey Global Institute director.


**Sanity check**

In [8]:
print(len(response.source_nodes))

2


In [9]:
# Print out each source node
print("Source nodes:")
print("=" * 50)

for i, node in enumerate(response.source_nodes):
    print(f"Node {i+1}:")
    print(f"Score: {node.score}")
    print(f"Text: {node.text}")
    print(f"Metadata: {node.metadata}")
    print("-" * 30)

Source nodes:
Node 1:
Score: 0.7128818159158221
Text: The state of AI  
March 2025
Alex Singla  
Alexander Sukharevsky  
Lareina Yee  
Michael Chui  
Bryce Hall
How organizations are rewiring to capture value
Metadata: {'page_label': '1', 'file_name': 'state_of_ai.pdf', 'file_path': '/content/drive/MyDrive/Colab/AI_Agents/state_of_ai.pdf', 'file_type': 'application/pdf', 'file_size': 5564174, 'creation_date': '2025-08-16', 'last_modified_date': '2025-08-15'}
------------------------------
Node 2:
Score: 0.7014180967894396
Text: McKinsey commentary
Lareina Yee
Senior partner and McKinsey Global Institute director
Although we remain in the early stages of gen AI, we’re beginning to get a glimpse into 
the ways the technology is affecting the workforce. A common fear about the technology 
is that it will be a job killer, as organizations offload tasks historically done by employees to 
increasingly powerful AI platforms. But our survey suggests that this is not necessarily the 
case. In f

In [10]:
# Ask more questions
response2 = query_engine.query("What are the main findings about AI adoption?")
print(response2)

The main findings about AI adoption are that the reported use of AI has been increasing steadily, with 78 percent of respondents stating their organizations use AI in at least one business function. The use of AI has been most commonly reported in IT and marketing/sales functions, with a notable increase in AI use in IT specifically. Additionally, organizations are now using AI in more business functions compared to previous surveys, with most respondents reporting AI use in more than one business function on average. However, less than one-third of respondents mentioned that their organizations are following most of the 12 adoption and scaling practices for gen AI.


In [11]:
response3 = query_engine.query("What does the document say about AI risks?")
print(response3)

The document mentions that organizations are actively managing risks related to inaccuracy, cybersecurity, and intellectual property infringement, which are some of the most common gen-AI-related risks causing negative consequences. Additionally, larger organizations are reported to be more likely to mitigate potential cybersecurity and privacy risks compared to other organizations.


# Agentic RAG

In [12]:
!pip install --upgrade datasets
!pip install --upgrade huggingface-hub

# Loading the data

In [13]:
from llama_index.core import SimpleDirectoryReader
from google.colab import drive
drive.mount('/content/drive')

# Path to your folder inside Google Drive
folder_path = '/content/drive/MyDrive/Colab/AI_Agents/state_of_ai.pdf'
# Load document
reader = SimpleDirectoryReader(input_files=[folder_path]) #input_files=["state_of_ai.pdf"]
documents = reader.load_data()
print(f"Loaded {len(documents)} document(s).")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 26 document(s).


**Breaking the data into chunks**

In [14]:
from llama_index.core.node_parser import SentenceSplitter

# chunk_size of 1024 is a good default value
splitter = SentenceSplitter(chunk_size=1024)
# Create nodes from documents
nodes = splitter.get_nodes_from_documents(documents)

In [15]:
for i, node in enumerate(nodes[:3]):  # Change 3 to any number of chunks you want to view
    print(f"\n--- Chunk {i+1} ---")
    print(node.get_content())



--- Chunk 1 ---
The state of AI  
March 2025
Alex Singla  
Alexander Sukharevsky  
Lareina Yee  
Michael Chui  
Bryce Hall
How organizations are rewiring to capture value

--- Chunk 2 ---
Organizations are beginning to create the 
structures and processes that lead to 
meaningful value from gen AI. While  
still in early days, companies are  
redesigning workflows, elevating  
governance, and mitigating  
more risks.   
O
rganizations are starting to make 
organizational changes designed to 
generate future value from gen AI, and 
large companies are leading the way. The 
latest McKinsey Global Survey on AI finds 
that organizations are beginning to take steps that drive 
bottom-line impact—for example, redesigning workflows as 
they deploy gen AI and putting senior leaders in critical roles, 
such as overseeing AI governance. The findings also show 
that organizations are working to mitigate a growing set of 
gen-AI-related risks and are hiring for new AI-related roles 
while they re

**Define the LLM and the Embedding Model**

In [16]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# LLM model
Settings.llm = OpenAI(model="gpt-3.5-turbo")
# embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

**Create the vector index and summary index**

In [17]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# summary index
summary_index = SummaryIndex(nodes)
# vector store index
vector_index = VectorStoreIndex(nodes)

**Create the vector query engine and summary query engine**

In [18]:
# summary query engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# vector query engine
vector_query_engine = vector_index.as_query_engine()

**Convert the vector and query engines into tools**

In [19]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the Topic and Context."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the the Topic and Context."
    ),
)

**Define a superset query engine (Router)**

In [20]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

**Test whether the query engine works**

In [21]:
response = query_engine.query("Who is Lareina Yee according to teh document?")
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the document, which would help in identifying who Lareina Yee is..
Lareina Yee is one of the authors listed in the document.


**Convert the query engine into a tool**

In [22]:
# Create tool wrapper around router
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="state_of_ai_report_assistant",
    description="Answers questions based on the McKinsey 2025 State of AI report.",
    return_direct=False,
)


** Define system prompt**

In [23]:
system_prompt = """
You are a helpful assistant specialized in answering questions using the 'State of AI' March 2025 report by McKinsey.
Your task is to:

1. Use the Summary Tool when the user asks for high-level insights, trends, survey findings, or general understanding
   (e.g., "What are the top AI adoption practices?" or "Summarize the report's key findings").

2. Use the Vector Tool when the user is asking for specific statistics, organizational practices, exhibit-based
   evidence, or detailed examples
   (e.g., "What percentage of companies track AI KPIs?" or "What are the risks companies are mitigating?").

Refer only to the content of the report. If the user's query is outside this context, politely decline or redirect.

Check your answer multiple times to make sure it is actually relevant and mentioned in the document.

Examples of summary queries:
- "How are companies restructuring to adopt GenAI?"
- "What does the report say about workforce reskilling?"

Examples of specific/vector queries:
- "What percentage of companies have a roadmap for GenAI adoption?"
- "Who is responsible for AI governance in large firms?"

Always explain clearly, referencing exact statistics, frameworks, or concepts when relevant. Be concise and insightful.
"""

# **Define the agent**

In [24]:

from llama_index.core.agent.workflow import AgentWorkflow

query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=Settings.llm,
    system_prompt=system_prompt,
)


# observability using Arize Phoenix

In [25]:
!pip install llama-index-callbacks-arize-phoenix arize-phoenix

In [26]:

!pip install --upgrade llama-index arize-phoenix llama-index-callbacks-arize-phoenix


  Using cached arize_phoenix-11.24.1-py3-none-any.whl.metadata (31 kB)
Using cached arize_phoenix-11.24.1-py3-none-any.whl (2.4 MB)
  Attempting uninstall: arize-phoenix
    Found existing installation: arize-phoenix 11.24.0
    Uninstalling arize-phoenix-11.24.0:
      Successfully uninstalled arize-phoenix-11.24.0


Run the agent and analyze responses

In [27]:
!pip install arize-phoenix-otel

In [28]:
!pip install 'arize-phoenix==11.24.0'

  Using cached arize_phoenix-11.24.0-py3-none-any.whl.metadata (31 kB)
Using cached arize_phoenix-11.24.0-py3-none-any.whl (2.4 MB)
  Attempting uninstall: arize-phoenix
    Found existing installation: arize-phoenix 11.24.1
    Uninstalling arize-phoenix-11.24.1:
      Successfully uninstalled arize-phoenix-11.24.1


In [29]:
import os

os.environ["PHOENIX_API_KEY"] = ""
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/s/junaeidshoaib"

# If you created your Phoenix Cloud instance before June 24th, 2025,
# you also need to set the API key as a header:
# os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.getenv('PHOENIX_API_KEY')}"

In [31]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/junaeidshoaib/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



/usr/local/lib/python3.11/dist-packages/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")


In [32]:
# In Jupyter/Colab, you can use await directly
question = "Who is Yareina Lee according to the document? Where is she mentioned in the document and in what context?"
response = await query_engine_agent.run(question)
print(response)

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the Topic and Context, which would be more suitable for finding information about 'Yareina Lee'..
Yareina Lee is not mentioned in the document. If you have any other questions or need information on a different topic from the report, feel free to ask!


In [33]:
!pip install llama-index-tools-arxiv llama-index-tools-wikipedia duckduckgo-search
!pip install llama-index-tools-brave-search


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=393a4ae01713d5acab1852a7b7e3ed22e470e1f48860dd50047d04ee90cf2afd
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=9af4f282681c4dc4de3d3365d8a6f3227ec5bb249f1a43db5635da3cd41a7013
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built wikipedia sgmllib3k


# Add the new tools (ArXiV, Brave Search)

In [43]:
# Import additional tools
from llama_index.tools.arxiv import ArxivToolSpec
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools import QueryEngineTool
from llama_index.tools.brave_search import BraveSearchToolSpec

import requests
import json


In [44]:
# Create ArXiV tool

arxiv_tool = ArxivToolSpec()

arxiv_tools = arxiv_tool.to_tool_list()

from google.colab import userdata

# Create Brave Search tool

brave_search_tool_spec = BraveSearchToolSpec(api_key=userdata.get('BraveSearch'))
brave_search_tools = brave_search_tool_spec.to_tool_list()


In [45]:

# Create enhanced agent with multiple tools - FIX: Use extend instead of append
enhanced_tools = [query_engine_tool]  # Start with McKinsey report tool
enhanced_tools.extend(brave_search_tools)  # Add all brave search tools
enhanced_tools.extend(arxiv_tools)  # Add all arxiv tools

## Define the enhanced agent with all tools

In [46]:
# Create new enhanced agent
enhanced_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=enhanced_tools,
    llm=Settings.llm,
    system_prompt="""You are an AI research assistant with access to:
    1. The McKinsey 2025 State of AI report
    2. Web search capabilities
    3. ArXiv research paper search

    Use these tools to provide comprehensive, well-researched answers. When discussing AI trends,
    combine insights from the McKinsey report with recent research and web findings.""",
)


# test agent with multiple questions!

In [47]:
# Test questions that can benefit from multiple tools

# Question 1: Combine McKinsey insights with recent research
question1 = """According to the McKinsey report, what are the main organizational changes companies are making for AI adoption?
Can you also search for recent research papers on AI governance and organizational transformation to provide additional context?"""

print("Question 1: Organizational changes and governance")
print("=" * 50)
response1 = await enhanced_agent.run(question1)
print(response1)
print("\n" + "="*80 + "\n")

Question 1: Organizational changes and governance
Selecting query engine 0: The question is asking for a summary of the main organizational changes companies are making for AI adoption, which aligns with the purpose of summarization questions related to the Topic and Context..
### Recent Research Papers on AI Governance:
1. **[AI Governance: Themes, Knowledge Gaps, and Future Agendas](https://www.emerald.com/insight/content/doi/10.1108/intr-01-2022-0042/full/html)** - This paper synthesizes the organizational AI governance literature by outlining research themes and knowledge gaps.
   
2. **[AI Governance: A Systematic Literature Review](https://link.springer.com/article/10.1007/s43681-024-00653-w)** - Presents a systematic literature review providing the state of the art on AI Governance, focusing on stakeholders and governance practices.

3. **[Responsible Artificial Intelligence Governance: A Review and Research Framework](https://www.sciencedirect.com/science/article/pii/S096386872

In [48]:
# Question 2: Workflow Redesign and Implementation
question2 = """What does the McKinsey report say about workflow redesign for AI implementation?
Search ArXiv for papers on business process automation with AI and find current web articles about workflow transformation."""

print("Question 2: Workflow Redesign")
response2 = await enhanced_agent.run(question2)
print(response2)


Question 2: Workflow Redesign
Selecting query engine 0: Workflow redesign for AI implementation involves summarizing the changes needed in the workflow to incorporate AI technology effectively..
### ArXiv Papers on Business Process Automation with AI:
1. **[D3BA: A Tool for Optimizing Business Processes Using Non-Deterministic Planning](http://arxiv.org/pdf/2001.02619v2):**
   - This paper introduces D3BA, a tool for optimizing business processes using AI planning. It focuses on composing services to automate subtasks within complex business processes.

2. **[Can Artificial Intelligence Transform DevOps?](http://arxiv.org/pdf/2206.00225v1):**
   - Explores the intersection of DevOps and AI, highlighting how AI can enhance DevOps processes such as testing, coding, releasing, monitoring, and system improvement.

3. **[Impact of Artificial Intelligence on Businesses](http://arxiv.org/pdf/1905.02092v1):**
   - Discusses the integration of AI in business processes, emphasizing the implicati

In [49]:
# Question 3: Risk management and future trends
question3 = """Based on the McKinsey report, what are the key risks organizations are addressing with gen AI?
Can you search the web for recent academic research on AI risk mitigation and compare with the report's findings?"""

print("Question 3: Risk management")
response3 = await enhanced_agent.run(question3)
print(response3)



Question 3: Risk management
Selecting query engine 0: Summarization questions related to the topic and context can help identify the key risks organizations are addressing with gen AI..
### Recent Academic Research on AI Risk Mitigation:

1. **[AI Risks that Could Lead to Catastrophe | CAIS](https://safe.ai/ai-risk)**
   - **Key Insight:** Proposes measures to mitigate risks from malicious use, particularly emphasizing biosecurity and strict access controls for AIs with biological research capabilities.

2. **[Artificial Intelligence Risk & Governance - Wharton Human-AI Research](https://ai.wharton.upenn.edu/white-paper/artificial-intelligence-risk-governance/)**
   - **Key Insight:** Providing explanations of how AI systems work, along with predictions, could aid in mitigating risks. Organizations are advised to share minimal information required by consumers or as mandated by law.

3. **[Managing extreme AI risks amid rapid progress | Science](https://www.science.org/doi/10.1126/scie

In [50]:
# Question that forces usage of all three tools
comprehensive_question = """Who is Lareina Yee in the McKinsey document and what are her views on AI's workforce impact?

After finding information about her from the document, please:
1. Search the web using Brave Search for recent articles, interviews, or news about Lareina Yee and her work on AI
2. Search ArXiv for any research papers she may have authored or co-authored related to AI, workforce transformation, or economic impact
3. Provide a comprehensive profile combining insights from all three sources about her expertise and contributions to AI research"""

print("Question: Comprehensive profile of Lareina Yee")
print("=" * 60)
print("This question should force the agent to use:")
print("1. Query Engine - to find info about Lareina Yee in the McKinsey document")
print("2. Brave Search - to find recent web articles/news about her")
print("3. ArXiv Search - to find any academic papers she's authored")
print("=" * 60)

response = await enhanced_agent.run(comprehensive_question)
print(response)

Question: Comprehensive profile of Lareina Yee
This question should force the agent to use:
1. Query Engine - to find info about Lareina Yee in the McKinsey document
2. Brave Search - to find recent web articles/news about her
3. ArXiv Search - to find any academic papers she's authored
Selecting query engine 1: The question is asking for specific context related to 'Lareina Yee', which aligns with the purpose of retrieving specific context from the Topic and Context..
### Lareina Yee Profile:

#### McKinsey Global Institute Director:
Lareina Yee is a Senior Partner and the Director of the McKinsey Global Institute. She is a global expert and leader of research on AI and frontier technologies, advising companies on growth and transformation.

#### Web Search Insights:
1. **McKinsey & Company Profile**: Lareina Yee is a thought leader at the intersection of technology and diversity. She chairs the McKinsey Technology Council and has contributed to key publications like the "Technology T